## Imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from read_data import *

classes_dict = {
    'by_country' : 4,
    'by_style' : 7,
    'by_product' : 2
}

## Set the classification problem

In [ ]:
problem_type = 'by_style' 

## Set saving / restoring

In [ ]:
restoring_mode = False
saving_mode = False

restoring_name = 'model.ckpt'
saving_name = 'model.ckpt'


restoring_path = os.path.join('models', problem_type, restoring_name)
saving_path = os.path.join('models', problem_type, saving_name)

## Adjust Hyperparameters

In [ ]:
EPOCHS = 1
BATCH_SIZE = 8
VALIDATION_BATCH = 16

IMG_SIZE = 150
CLASSES = classes_dict[problem_type]

## Neural Network functions

### Convolution Layers

In [ ]:
def conv_layer(X, filters, filter_size, name, activation=None):
    """Create a new convolution layer with Xavier initializer"""
    
    with tf.variable_scope(name):
        
        # create Xavier initializer node 
        in_channels = int(X.get_shape()[3])
        init = tf.contrib.layers.xavier_initializer_conv2d()
    
        # create the parameter structures         
        W = tf.get_variable(initializer=init, 
                            shape=(filter_size[0], filter_size[1],
                                   in_channels, filters),
                            name="weights")
        b = tf.get_variable(initializer=tf.zeros(filters),
                            name="biases")
        
        # perform convolution and add bias
        conv = tf.nn.conv2d(X, W, strides=(1, 1, 1, 1), padding="SAME")
        z = tf.nn.bias_add(conv, b)
        
        # activation function
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

        
def pooling_layer(X, kernel_size):
    """Perform max pooling"""
    
    return tf.nn.max_pool(X,
                          ksize=(1, kernel_size[0], kernel_size[1], 1),
                          strides=(1, kernel_size[0], kernel_size[1], 1),
                          padding="VALID")

### Dense Layers

In [ ]:
def dense_layer(X, n_neurons, name, activation=None):
    """Create a new fully connected layer with Xavier initializer"""
    
    with tf.variable_scope(name):
        
        # create Xavier initializer node 
        n_inputs = int(X.get_shape()[1])
        init = tf.contrib.layers.xavier_initializer()
        
        # create the parameter structures     
        W = tf.get_variable(initializer=init,
                            shape=(n_inputs, n_neurons),
                            name="weights")
        b = tf.get_variable(initializer=tf.zeros(n_neurons),
                            name="biases")
        z = tf.matmul(X, W) + b
        
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

## Build the model and deploy it on a device

In [ ]:
with tf.device('/cpu:0'):
    
    #==================[ READ AND PROCESS THE INPUT ]==================#
    
    # decide the dataset input type
    is_training = tf.placeholder(tf.bool, name="is_training")
    
    # load training data from input queues     
    images_trn, labels_trn = inputs(problem_type, BATCH_SIZE, EPOCHS)
    
    # load validation data from feed dictionary
    images_val = tf.placeholder(tf.uint8, shape=[VALIDATION_BATCH, IMG_SIZE, IMG_SIZE, 3])
    labels_val = tf.placeholder(tf.int32, shape=[VALIDATION_BATCH,])
    
    # choose the input
    images = tf.cond(is_training, lambda: images_trn, lambda: images_val)
    labels = tf.cond(is_training, lambda: labels_trn, lambda: labels_val)

    # normalize the images     
    images = (tf.cast(images, tf.float32) / 255.0)
    

In [ ]:
with tf.device('/gpu:0'):

    #==================[ CONVOLUTIONAL LAYERS ]==================#
    
    images_conv_11 = conv_layer(images, 32, (5, 5), "conv_11", "relu")
    images_conv_12 = conv_layer(images_conv_11, 32, (5, 5), "conv_12", "relu")
    images_pool_1  = pooling_layer(images_conv_12, (2, 2))
    
    images_conv_21 = conv_layer(images_pool_1, 64, (3, 3), "conv_21", "relu")
    images_conv_22 = conv_layer(images_conv_21, 64, (3, 3), "conv_22", "relu")
    images_pool_2  = pooling_layer(images_conv_22, (2, 2))
    
    images_conv_31 = conv_layer(images_pool_2, 64, (3, 3), "conv_31", "relu")
    images_conv_32 = conv_layer(images_conv_31, 64, (3, 3), "conv_32", "relu")
    images_pool_3  = pooling_layer(images_conv_32, (2, 2))
    
    #==================[     DENSE LAYERS     ]==================#
    
    images_flatten = tf.contrib.layers.flatten(images_pool_3)
    images_dense_1 = dense_layer(images_flatten, 256, "dense_1", "relu")
    images_dense_2 = dense_layer(images_dense_1, 256, "dense_2", "relu")
    
    #==================[     OUTPUT LAYER     ]==================#
    
    logits = dense_layer(images_dense_2, CLASSES, "logits")
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
    #==================[     OPTIMIZATION     ]==================#
    
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    training_op = optimizer.minimize(loss)
    

## Create the session and start the threads for input queues

In [ ]:
# create the session saver
saver = tf.train.Saver()

# create a session for running operations in the graph.
sess = tf.Session()

# create the variable initializers
init_op = tf.group(tf.global_variables_initializer(),
                   tf.local_variables_initializer())

# initialize the variables
sess.run(init_op)

if restoring_mode:
    # previously saved model is restored
    saver.restore(sess, restoring_path)
    
# start input enqueue threads.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

## Collect data for Tensorboard

In [ ]:
with tf.device('/cpu:0'):
    
    tf.summary.image('conv_12', images_conv_12[:,:,:,:3])
    tf.summary.image('conv_22', images_conv_22[:,:,:,:3])
    tf.summary.image('conv_32', images_conv_32[:,:,:,:3])
    tf.summary.scalar('loss', loss)

    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(
        os.path.join('tensorboard', problem_type), sess.graph)

## Load validation data

In [ ]:
# load validation data
images_validation = np.load(os.path.join('data_' + problem_type, 'testing_data.dat'))
labels_validation = np.load(os.path.join('data_' + problem_type, 'testing_labels.dat'))

## Training loop

In [ ]:
try:
    step = 0
    
    # feed data until the epoch limit is reached     
    while not coord.should_stop():

        step += 1
        
        _, loss_value, summary = sess.run([training_op, loss, merged], feed_dict={
            is_training : True,
            images_val : images_validation[:16],
            labels_val : labels_validation[:16]
        })
        
        # display status once in a while         
        if step % 10 == 0:
            print("Step {0} : loss = {1:.2f}".format(step, loss_value), flush=True)
            train_writer.add_summary(summary, step)
        
        saving_condition = True
        
        # save the model for later use         
        if saving_mode and saving_condition:
            saver.save(sess, saving_path)
        
        
except tf.errors.OutOfRangeError:
    
    print('\nDone training -- epoch limit reached\n')
    
finally:
    
    # when done, ask the threads to stop
    coord.request_stop()

    # wait for threads to finish
    coord.join(threads)
    sess.close()